# Algorithms for Data Science

## Finding Similar Items

The objective of this lab is to implement the Min-Hashing and Locality Sensitive Hashing. This lab needs Python and Jupyter, along with the NumPy package.

1. We first load the required libraries and the files containing the documents.

In [100]:
import sys
import numpy as np
import urllib.request
import re
import string
import random

file_location = 'https://www.lri.fr/~maniu/tweets.txt' #you can change this to a local file on your computer

#keeping document in memory
infile = urllib.request.urlopen(file_location)
docs = []

for line in infile: 
  docs.append(str(line.strip()).lower())
print("Number of documents: %d"%len(docs))
#print(docs)


Number of documents: 497


2. We transform the document into $k$-shingles, and we hash them to their integer values. We compute the Jaccard similarity between two documents given as sets of shingle ids.

In [101]:
k = 5 #k for shingles

shingle_id = {}
id_shingle = []
m = [] # matrix of the document (hash table)
ids = 0 # the order of the unique shingle_id

total_shingles = 0

for d in docs:
  #removing whitespace
  d_new = ''.join(c for c in d if c.isalnum())
  char_shing = [d_new[i:i+k] for i in range(len(d_new)-k+1)]
  total_shingles += len(char_shing)
  sid = set()
  for sh in char_shing:
    if sh not in shingle_id:
      shingle_id[sh]=ids
      id_shingle.append(sh)
      ids=ids+1
    sid.add(shingle_id[sh])
  m.append(sid)

print ("Unique shingles: %d"%len(id_shingle))
print ("Total shingles: %d"%total_shingles)
  

Unique shingles: 19532
Total shingles: 28150


In [5]:
print(len(m))

497


In [102]:
def jaccard_similarity(doc1, doc2):
  if len(doc1)==0 or len(doc2)==0:
    return 0.0
  else:
    inter = doc1.intersection(doc2)
    union = doc1.union(doc2)
    return float(len(inter))/float(len(union))

#example

print(jaccard_similarity(m[0],m[1]))

0.043859649122807015


3. We implement the method for min-hashing given a permutation.

In [119]:
def min_hash(doc,perm):
  for d in perm:
    if d in doc: 
          return d  # return the first appear signature
    
perm = list(range(len(id_shingle)))
random.shuffle(perm)
#print(perm)
min_hash(m[0],perm)
print(len(m))
print(len(id_shingle))

497
19532


In [120]:
min_hash(m[0],perm)

9

4. Implement the full Min-Hashing signature matrix for a given number $n$ of permutations. Implement the similarity estimation based on Min-Hashing (i.e., the number of permutation on which two documents agree).

In [87]:
# YOUR CODE HERE

def min_hash_sim(lst1,lst2):
    assert(len(lst1)==len(lst2))
    agree=0
    for i in range(len(lst1)):
        if (lst1[i]==lst2[i]):
            agree=agree+1
    return agree/len(lst1)

# generate several permutations
n=100
p=[]
for _ in range(n):
    perm = list(range(len(id_shingle)))
    random.shuffle(perm)
    p.append(perm)
sig=[]

for doc in m:
    h=[] # documents
    for per in p:
        hval=min_hash(doc,per)
        h.append(hval)
    sig.append(h) # signature matrix


5. __TASK__ Implement Locality-Sensitive Hashing, given $b$ bands of $r$ rows such that $br=n$. Compute the similarity threshold needed using the formula in the lecture $t=(1/b)^{1/r}$. Assume that signatures in the same band are similar only if the are the same (i.e., they agree on all columns). Check for similarity all documents that agree in at least one band, and compare with the true jaccard similarity.

## Situation 1: r=2 b=50

In [140]:
# YOUR CODE HERE

#1. define the similarity threshold s, find b and r
#b*r=100
r=2
b=int(n/r)
s=(1/b)**(1/r)

chunks = [[s[x:x+r] for x in range(0, b*r, r)] for s in sig]
#2. each document is an array of length n , you have to devide it into b bands of r values
bands= [{}for i in range (b)]
for i in range(len(bands)):
    for doc_id,chunk in enumerate(chunks):
        
        key ='_'.join(map(str, chunk[i]))
        if key not in bands[i]:
            bands[i][key]=[]  # add the unique hash key 
        bands[i][key].append(doc_id) 
#print (bands[0])           
print (len([band for band in bands if len([value for value in band.values() if len(value)>1])]))
#3. have a dictionary per band: keys are the hashes of the partial signatures, values are list of document ids

#4. for each document, divide it into b slices ( b slices of r value), one possible has [1,5,4,2]
#b=2,r=2 hash1="1_5" hash2="4_2";add the document id to the list (in the dict) corresponding to the hash
#5. loop over the values in the dict; retrive list in each key; do pairwise comparaison; reject sim<s


50


### Find the similarity pair of the documents compare jaccard similarity with threshold

In [141]:
sim_docs = {}
for band in bands: 
    for key, value in band.items():
        if len(value)>1:
            for doc1,doc2,sim in [(x,y,jaccard_similarity(m[x], m[y])) for i,x in enumerate(value) for j,y in enumerate(value) if i != j]:
                if sim>s and f'{doc1}/{doc2}' not in sim_docs and f'{doc2}/{doc1}' not in sim_docs:
                    sim_docs[f'{doc1}/{doc2}'] = sim

### Print the similar document pair

In [142]:
print(len(sim_docs.keys()))
for key in sim_docs.keys():
    print(key)

182
1/237
10/12
45/283
46/284
52/53
52/288
53/288
124/131
169/408
183/423
61/303
109/110
109/352
110/352
124/363
131/363
129/370
475/477
78/319
8/62
8/303
62/303
45/47
45/281
45/284
47/281
47/284
281/284
61/63
244/245
10/411
58/290
303/304
303/305
304/305
107/348
124/125
124/128
124/129
124/362
124/365
124/368
124/371
125/128
125/129
125/362
125/363
125/365
125/368
125/371
126/128
126/129
128/129
128/362
128/363
128/365
128/368
128/371
129/362
129/363
129/365
129/368
129/371
362/363
362/365
362/368
362/371
363/365
363/368
363/371
365/368
365/371
368/371
51/53
51/288
124/361
124/370
125/361
125/370
361/363
363/370
368/370
128/372
406/409
108/109
108/110
108/352
124/132
125/132
129/132
132/362
132/363
362/370
71/311
84/311
310/311
126/372
128/134
152/391
19/252
86/328
129/130
129/135
129/364
130/135
130/370
135/370
125/372
55/294
40/278
61/62
61/300
62/63
62/300
63/300
109/111
111/352
125/131
131/371
365/366
109/351
110/111
111/351
351/352
127/367
188/190
283/284
175/421
108/111
108/351


When we choose r=2 we have 182 pair that is similar

## Situation2: r=5 b=20

In [133]:
# YOUR CODE HERE

#1. define the similarity threshold s, find b and r
#b*r=100
r=5
b=int(n/r)
s=(1/b)**(1/r)

chunks = [[s[x:x+r] for x in range(0, b*r, r)] for s in sig]

bands= [{}for i in range (b)]
for i in range(len(bands)):
    for doc_id,chunk in enumerate(chunks):
        
        key ='_'.join(map(str, chunk[i]))
        if key not in bands[i]:
            bands[i][key]=[]  # add the unique hash key 
        bands[i][key].append(doc_id) 
#print (bands[0])           
print (len([band for band in bands if len([value for value in band.values() if len(value)>1])]))


8


In [137]:
sim_docs = {}
for band in bands: 
    for key, value in band.items():
        if len(value)>1:
            for doc1,doc2,sim in [(x,y,jaccard_similarity(m[x], m[y])) for i,x in enumerate(value) for j,y in enumerate(value) if i != j]:
                if sim>s and f'{doc1}/{doc2}' not in sim_docs and f'{doc2}/{doc1}' not in sim_docs:
                    sim_docs[f'{doc1}/{doc2}'] = sim

In [138]:
for key in sim_docs.keys():
    print(key)

124/363
244/245
110/352


When we choose a bigger r we have three pair that is similar

To choose an ideal r, we need to plot the s curve and see the surface of the false postive and false negatives. The idea is to minimize them both.  